In [1]:
pip install nibabel matplotlib numpy

Note: you may need to restart the kernel to use updated packages.


# Segmentation Analysis

In [2]:
import os
import nibabel as nib
import numpy as np
import csv

# Population directory
base_dir = "/kaggle/input/bratsmen"
output_csv = "meningioma_analysis.csv"

# Prepare a CSV file to save the results
with open(output_csv, mode="w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    # Write the header row
    writer.writerow([
        "File", "Shape", "Background Percentage", "Label 1 Percentage",
        "Label 2 Percentage", "Label 3 Percentage", "Missing Labels"
    ])
    
    # Loop through each subfolder
    for subfolder in os.listdir(base_dir):
        seg_file = os.path.join(base_dir, subfolder, f"{subfolder}-seg.nii")
        if os.path.exists(seg_file):
            # Load segmentation file
            seg_data = nib.load(seg_file)
            seg_array = seg_data.get_fdata()
            
            # Get the shape of the segmentation mask
            seg_shape = seg_array.shape
            
            # Find unique values and their counts
            unique_values, counts = np.unique(seg_array, return_counts=True)
            total_voxels = np.prod(seg_array.shape)
            
            # Calculate percentages for each label
            label_counts = dict(zip(unique_values, counts))
            total_tumor_voxels = sum(label_counts.get(label, 0) for label in [1, 2, 3])
            
            percentages = {
                "background": (label_counts.get(0, 0) / total_voxels) * 100,
                1: (label_counts.get(1, 0) / total_tumor_voxels) * 100 if total_tumor_voxels > 0 else 0,
                2: (label_counts.get(2, 0) / total_tumor_voxels) * 100 if total_tumor_voxels > 0 else 0,
                3: (label_counts.get(3, 0) / total_tumor_voxels) * 100 if total_tumor_voxels > 0 else 0,
            }
            
            # Check for missing labels
            expected_labels = {1, 2, 3}
            present_labels = set(label_counts.keys())
            missing_labels = expected_labels - present_labels
            
            # Print results
            print(f"Segmentation file: {subfolder}")
            print(f"  Shape: {seg_shape}")
            print(f"  Background Percentage: {percentages['background']:.2f}%")
            print(f"  Label 1 Percentage (Subtumor 1): {percentages[1]:.2f}%")
            print(f"  Label 2 Percentage (Subtumor 2): {percentages[2]:.2f}%")
            print(f"  Label 3 Percentage (Subtumor 3): {percentages[3]:.2f}%")
            if missing_labels:
                print(f"  Missing Labels: {missing_labels}")
            else:
                print("  No missing labels")
            print("-" * 50)
            
            # Write results to CSV
            writer.writerow([
                subfolder, seg_shape, f"{percentages['background']:.2f}%",
                f"{percentages[1]:.2f}%", f"{percentages[2]:.2f}%", f"{percentages[3]:.2f}%",
                ", ".join(map(str, missing_labels)) if missing_labels else "None"
            ])


Segmentation file: BraTS-MEN-00782-000
  Shape: (240, 240, 155)
  Background Percentage: 99.99%
  Label 1 Percentage (Subtumor 1): 0.00%
  Label 2 Percentage (Subtumor 2): 0.00%
  Label 3 Percentage (Subtumor 3): 100.00%
  Missing Labels: {1, 2}
--------------------------------------------------
Segmentation file: BraTS-MEN-00495-000
  Shape: (240, 240, 155)
  Background Percentage: 99.98%
  Label 1 Percentage (Subtumor 1): 0.00%
  Label 2 Percentage (Subtumor 2): 0.00%
  Label 3 Percentage (Subtumor 3): 100.00%
  Missing Labels: {1, 2}
--------------------------------------------------
Segmentation file: BraTS-MEN-00247-000
  Shape: (240, 240, 155)
  Background Percentage: 99.94%
  Label 1 Percentage (Subtumor 1): 0.00%
  Label 2 Percentage (Subtumor 2): 0.00%
  Label 3 Percentage (Subtumor 3): 100.00%
  Missing Labels: {1, 2}
--------------------------------------------------
Segmentation file: BraTS-MEN-01357-000
  Shape: (240, 240, 155)
  Background Percentage: 99.60%
  Label 1 Per